In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn
from sklearn.preprocessing import scale

from sklearn.model_selection import train_test_split, KFold
from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score

## Useful Functions

In [2]:
def check_state(val, theta):
    if val >= theta: return 1
    else: return 0


def compare_result(result, truth):
    if truth == 1:
        if result == 1: return 1
        elif result == 0: return 2
    elif truth == 0:
        if result == 1: return 3
        elif result == 0: return 4


def estimation_measures(TP, FN, FP, TN):
    if TP == 0 and FP == 0:
        precision = 0
    else:
        precision = TP/(TP + FP)

    if TP == 0 and FN == 0:
        recall = 0
    else:
        recall = TP/(TP + FN)

    if precision == 0 and recall == 0:
        f1_score = 0
    else:
        f1_score = 2*precision*recall/(precision + recall)
    return f1_score



def check_estimation(y_estimate, y_truth, theta):
    TP = 0
    TN = 0
    FN = 0
    FP = 0
    for i in range(len(y_estimate)):
        estimated = y_estimate[i]
        truth = y_truth[i]
        result = check_state(estimated, theta)
        score_state = compare_result(result, truth)
        if score_state == 1: TP += 1
        elif score_state == 2: FN += 1
        elif score_state == 3: FP += 1
        elif score_state == 4: TN += 1
    data = estimation_measures(TP, FN, FP, TN)
    return data


In [3]:
def get_optimal_theta(est_val, ground_val):
    theta_values = [float("{0:.2f}".format((0.0 + 0.02*i))) for i in range(50)]
    f1_score = []
    for theta in theta_values:
        f1 = check_estimation(est_val, ground_val, theta)
        f1_score.append(f1)
    
    index = 0
    for i in range(1, len(theta_values)):
        if f1_score[index] < f1_score[i]:
            index = i
#     print(theta_values[index], f1_score[index])
#     plt.plot(theta_values, f1_score)
#     plt.show()
    return theta_values[index]


def get_predicted_value(X, theta):
    pred = []
    for i in range(len(X)):
        if X[i] < theta:
            pred.append(0)
        else:
            pred.append(1)
    return pred

In [4]:
def print_results(result):
    print("Precision: %.1f" % result[0])
    print("Recall: %.1f" % result[1])
    print("F1 Score: %.1f" % result[2])
    print("Area Under Curve: %.1f" % result[3])
    

def evaluation_results(y_test, y_predict):
    recall = recall_score(y_test, y_predict)*100
    precision = precision_score(y_test, y_predict)*100
    f1 = f1_score(y_test, y_predict)*100
    auc = roc_auc_score(y_test, y_predict)*100
    return [precision, recall, f1, auc]


def k_fold_testing(X, Y):
    results = []
    kf = KFold(n_splits = 5)
    kf.get_n_splits(X)
    for train_index, test_index in kf.split(X):
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        opt_theta = get_optimal_theta(x_train, y_train)
        y_predict = get_predicted_value(x_test, opt_theta)
        result = evaluation_results(y_test, y_predict)
        results.append(result)
    results = np.array(results)
    final_result = np.mean(results, axis = 0)
    return final_result

## Results - Proposed Method

In [5]:
df_propose = pd.read_csv("predicted_prereq.csv")
X_propose1 = df_propose[["tfidf_score"]].to_numpy().ravel()
X_propose2 = df_propose[["wiki_tfidf_score"]].to_numpy().ravel()
y_propose = df_propose[["relation"]].to_numpy().ravel()

In [8]:
# Proposed book tfidf
result = k_fold_testing(X_propose1, y_propose)
print_results(result)

Precision: 62.2
Recall: 49.3
F1 Score: 54.9
Area Under Curve: 67.9


In [9]:
# Proposed book + wikipedia tfidf
result = k_fold_testing(X_propose2, y_propose)
print_results(result)

Precision: 31.2
Recall: 100.0
F1 Score: 47.5
Area Under Curve: 50.0


## Results - RefD Method

In [8]:
df_propose = pd.read_csv("refd_estimated_results.csv")
X_propose1 = df_propose[["refd_equal"]].to_numpy().ravel()
X_propose2 = df_propose[["refd_tfidf"]].to_numpy().ravel()
y_propose = df_propose[["relation"]].to_numpy().ravel()

In [9]:
# RefD with Equal Method
result = k_fold_testing(X_propose1, y_propose)
print_results(result)

Precision: 50.6
Recall: 71.5
F1 Score: 59.1
Area Under Curve: 69.8


In [10]:
# RefD with TFIDF Method
result = k_fold_testing(X_propose2, y_propose)
print_results(result)

Precision: 39.3
Recall: 80.1
F1 Score: 52.7
Area Under Curve: 62.1


## Results - RefD Given Vals

In [5]:
df_propose = pd.read_csv("refd_given_vals.csv")
X_propose1 = df_propose[["refd_val"]].to_numpy().ravel()
# X_propose2 = df_propose[["refd_tfidf"]].to_numpy().ravel()
y_propose = df_propose[["output"]].to_numpy().ravel()

In [7]:
# RefD with Equal Method
result = k_fold_testing(X_propose1, y_propose)
print_results(result)

Precision: 51.0
Recall: 74.3
F1 Score: 60.1
Area Under Curve: 70.7
